In [1]:
import os
import zipfile
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
df_list = []

# 데이터프레임 생성
for dataset in os.listdir('../dataset'):
    if dataset.endswith('.csv'):
        globals()[dataset[:-4]] = pd.read_csv(f'../dataset/{dataset}')
        print(dataset[:-4])
        df_list.append(dataset[:-4])

products
orders
order_products__train
departments
aisles
order_products__prior
sample_submission


user_id, product_id, 피드백(reordered)

reordered
user_id : 무작위 행렬(잠재요인)1 
shape(user_id, 잠재요인의 갯수)
product_id : 무작위 행렬(잠재요인)2
shape(useproduct_idr_id, 잠재요인의 갯수)

내적 => reordered의 예측값

user1, product1의 잠재요인의 내적 0.1
실제로는 1
차이가 0.9

차이가 0.8
차이가 0.7
1

잠재요인 행렬 1,2
왜? 이게 feature가 됨

기존고객, 기존 상품들에 대해서 추천하기 좋다(피쳐도 자동으로 만들어 주니까 좋드라)


기준 -> 무작위 행렬(잠재요인)

In [3]:
orders_products = pd.concat([order_products__prior, order_products__train]).reset_index(drop=True)
orders_products
# order_prudocts__test

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
33819101,3421063,14233,3,1
33819102,3421063,35548,4,1
33819103,3421070,35951,1,1
33819104,3421070,16953,2,1


In [4]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [5]:
len(orders[orders['eval_set']=='train']) / len(orders[orders['eval_set']=='prior'])

0.040813108072042636

In [6]:
# 재구매에 대한 예측을 하기 위해 user의 order_number가 1인 row는 드랍
orders = orders.dropna()
len(orders[orders['eval_set']=='train'])/len(orders[orders['eval_set']=='prior'])



0.04361037204208511

In [7]:
o_train = pd.merge(orders[['order_id', 'user_id', 'eval_set']], orders_products[['order_id', 'product_id', 'reordered']], on='order_id')
o_train

,order_id,user_id,eval_set,product_id,reordered
0,2398795,1,prior,196,1
1,2398795,1,prior,10258,0
2,2398795,1,prior,12427,1
3,2398795,1,prior,13176,0
4,2398795,1,prior,26088,1
...,...,...,...,...,...
31741033,272231,206209,train,40603,0
31741034,272231,206209,train,15655,0
31741035,272231,206209,train,42606,0
31741036,272231,206209,train,37966,0


In [8]:
train_df = o_train[o_train['eval_set']=='prior'].drop(columns=['order_id', 'eval_set'])
test_df = o_train[o_train['eval_set']=='train'].drop(columns=['order_id', 'eval_set'])

In [9]:
from scipy import sparse

# 유저가 아이템을 구매한 횟수를 confidence (c_ui)값로 사용하여 CSR 희소 행렬 생성
sparse_user_item = sparse.csr_matrix((train_df['reordered'].astype(float),
                                      (train_df['user_id'], train_df['product_id'])))
sparse_user_item

<206210x49689 sparse matrix of type '<class 'numpy.float64'>'
	with 12439884 stored elements in Compressed Sparse Row format>

In [10]:
import implicit

# ALS 모델 인스턴스 생성
model = implicit.als.AlternatingLeastSquares(
    factors=128,                 # 잠재 요인의 수
    regularization=1e-4,         # 정규화 파라미터
    iterations=20,             # 학습 반복 횟수
    calculate_training_loss=True,  # 학습 손실 계산 여부
    use_gpu=False               # GPU 사용 여부
)

/opt/anaconda3/envs/EDA/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/EDA/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [11]:
model.fit(sparse_user_item)

100%|██████████| 20/20 [04:34<00:00, 13.73s/it, loss=0.0011]


In [12]:
# User Matrix와 Item Matrix의 크기 확인
user_vecs = model.user_factors  # User Matrix
item_vecs = model.item_factors  # Item Matrix

# 각 행렬의 차원(크기) 출력
print(user_vecs.shape)
print(item_vecs.shape)

(206210, 50)
(49689, 50)


In [13]:
# 정밀도와 재현율 계산 함수
def get_precision(relevant, recommend):
    _intersection = set(recommend).intersection(set(relevant))
    return len(_intersection) / len(recommend)

def get_recall(relevant, recommend):
    _intersection = set(recommend).intersection(set(relevant))
    return len(_intersection) / len(relevant)

In [16]:
%%time
# 특정 사용자(여기서는 사용자 0)에게 추천할 아이템 10개 선정 (filter_already_liked_items=False)
recommended_items, scores = model.recommend(1, sparse_user_item[1], N=10, filter_already_liked_items=False)

# 추천 결과와 점수 출력
recommended_results = [(item, score) for item, score in zip(recommended_items, scores)]
print(recommended_results)

[(13176, 0.8573645), (196, 0.70200145), (6184, 0.4935932), (49235, 0.4910095), (37710, 0.4844001), (38928, 0.4665045), (46149, 0.4586396), (43154, 0.4393425), (31651, 0.43527392), (41400, 0.4065036)]
CPU times: user 60.9 ms, sys: 41 ms, total: 102 ms
Wall time: 54 ms


In [17]:
get_precision(o_train[(o_train['user_id']==1) & (o_train['eval_set']=='train')]['product_id'], recommended_items)

0.4

In [19]:
order_id = {'order_id' : []}
order_id['order_id'].append(1)
order_id['order_id']

[1]

In [25]:
from tqdm import tqdm

# 추천할 아이템의 수를 설정합니다.
k = 11

# 정밀도, 재현율, NDCG를 저장할 리스트를 초기화합니다.
precisions, recalls = [], []
result = {
    'order_id' : [],
    'products_id' : []}

# Test 데이터셋의 각 사용자에 대해 반복합니다.
for user_id, user_df in tqdm(test_df.groupby('user_id')):
    # 해당 사용자가 구매한 아이템 목록을 추출합니다.
    buy_items = list(user_df['product_id'])

    # result buy_items를 추가합니다.
    result['order_id'].append(user_id)
    result['products_id'].append(buy_items)


    # 추천할 아이템을 결정합니다.
    recommended_items, scores = model.recommend(user_id, sparse_user_item[user_id], N=k, filter_already_liked_items=False)
    top_k_item =recommended_items

    # 정밀도, 재현율, NDCG를 계산하여 리스트에 추가합니다.
    precisions.append(get_precision(buy_items, top_k_item))
    recalls.append(get_recall(buy_items, top_k_item))

# 각 지표의 평균 값을 출력합니다.
print(f'\nprecision@k: {np.mean(precisions)}')
print(f'recall@k: {np.mean(recalls)}')


100%|██████████| 131209/131209 [01:44<00:00, 1254.42it/s]



precision@k: 0.1317634114622126
recall@k: 0.161534008126048


In [26]:
F1 = 2 * (np.mean(precisions) * np.mean(recalls)) / (np.mean(precisions) + np.mean(recalls))
F1

0.1451378059018203

In [28]:
result

{'order_id': [1,
  2,
  5,
  7,
  8,
  9,
  10,
  13,
  14,
  17,
  18,
  21,
  23,
  24,
  27,
  29,
  30,
  34,
  37,
  38,
  41,
  42,
  43,
  44,
  46,
  47,
  48,
  49,
  50,
  52,
  53,
  55,
  56,
  59,
  62,
  63,
  64,
  65,
  66,
  67,
  70,
  71,
  72,
  74,
  76,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  86,
  87,
  88,
  89,
  90,
  91,
  93,
  95,
  96,
  97,
  99,
  102,
  103,
  104,
  105,
  106,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  123,
  124,
  125,
  128,
  130,
  131,
  132,
  135,
  138,
  139,
  140,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  150,
  155,
  157,
  160,
  161,
  163,
  164,
  167,
  169,
  171,
  173,
  174,
  175,
  176,
  178,
  179,
  183,
  184,
  185,
  187,
  189,
  191,
  192,
  193,
  195,
  197,
  198,
  199,
  201,
  203,
  204,
  205,
  206,
  208,
  209,
  210,
  211,
  212,
  213,
  214,
  215,
  217,
  218,
  219,
  220,
  222,
  223,
  224,
  226,
  229,
  230,


In [31]:
user_df

,user_id,product_id,reordered
31741030,206209,6846,1
31741031,206209,9405,1
31741032,206209,24852,1
31741033,206209,40603,0
31741034,206209,15655,0
31741035,206209,42606,0
31741036,206209,37966,0
31741037,206209,39216,1


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import pandas as pd

# Load your dataset
# For this example, let's assume you have a CSV file
data = pd.read_csv('your_dataset.csv')

# Specify the feature columns and the target column
features = data.drop(columns=['target'])  # Replace 'target' with your actual target column name
target = data['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Set the parameters for LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',  # Change to 'binary' or 'multiclass' for classification
    'metric': 'rmse',           # Change to 'binary_logloss' or 'multi_logloss' for classification
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
num_boost_round = 100
bst = lgb.train(params, train_data, num_boost_round, valid_sets=[test_data], early_stopping_rounds=10)

# Make predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

  Using cached lightgbm-4.3.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
anceled
